In [ ]:
import pandas as pd
import numpy as np
import re

In [ ]:
columns_to_read=['DocID','Case_Name', 'Judgement_Date','Author','Bench', 'CaseID/CitationID','Verdict','IPC','CRPC','CPC','Acts','PrecedentCaseID','Precedent','CommonIPC','CommonCRPC','CommonCPC','CommonActs','Location']
data = pd.read_csv("legal_data.csv",encoding='latin-1', usecols=columns_to_read)
data = data.dropna(how='all')

data.head()

In [ ]:
!pip install py2neo

In [ ]:
from py2neo import Graph,Node,Relationship
import math
import ast

# Connect to your Neo4j instance bolt+s://<your-neo4j-aura-instance>.databases.neo4j.io:7687

graph = Graph("<connection uri>", auth=("neo4j", "<pass>"))

# Create a dictionary to store case nodes by CaseID for quick access
case_nodes = {}

# Iterate through the DataFrame and create case nodes
for index, row in data.iterrows():
    if math.isnan(row['DocID']):
      continue
    doc_id = int(row['DocID'])
    print(doc_id)

    ipc_list = list()
    crpc_list = list()
    cpc_list = list()
    acts_list = list()

    if type(row['IPC']) == str:
      ipc_list = ast.literal_eval(row['IPC'])

    if type(row['CRPC']) == str:
      crpc_list = ast.literal_eval(row['CRPC'])

    if type(row['CPC']) == str:
      cpc_list = ast.literal_eval(row['CPC'])

    if type(row['Acts']) == str:
      acts_list = ast.literal_eval(row['Acts'])

    case_node = Node("Case",
                     CaseID=doc_id,
                     Case_Name=str(row['Case_Name']).strip(),
                     Judgement_Date=str(row['Judgement_Date']).strip(),
                     Author=str(row['Author']).strip(),
                     Bench=str(row['Bench']).split(","),
                     CaseID_CitationID=str(row['CaseID/CitationID']).split(","),
                     Verdict=str(row['Verdict']).strip(),
                     IPC=ipc_list,
                     CRPC=crpc_list,
                     CPC=cpc_list,
                     Acts=acts_list,
                     Location=str(row['Location']))

    # Merge nodes in the Neo4j database
    graph.merge(case_node, "Case", "CaseID")

    # Store the case node in the dictionary for quick access
    case_nodes[doc_id] = case_node

count = 0
# Iterate through the DataFrame again to create CITES relationships
for _, row in data.iterrows():
  try:
    if math.isnan(row['PrecedentCaseID']):
        continue
    doc_id = int(row['DocID'])
    precedent_id = int(str(row['PrecedentCaseID']).split('.')[0].replace("\"","").strip())
    # precedent_id = int(row['PrecedentCaseID'].strip().replace("\"",""))

    # Get the case nodes from the dictionary
    case_node = case_nodes[doc_id]
    precedent_node = case_nodes[precedent_id]

    common_ipc_list = list()
    common_crpc_list = list()
    common_cpc_list = list()
    commomn_acts_list = list()

    if type(row['CommonIPC']) == str:
      common_ipc_list = ast.literal_eval(row['CommonIPC'])

    if type(row['CommonCRPC']) == str:
      common_crpc_list = ast.literal_eval(row['CommonCRPC'])

    if type(row['CommonCPC']) == str:
      common_cpc_list = ast.literal_eval(row['CommonCPC'])

    if type(row['CommonActs']) == str:
      common_acts_list = ast.literal_eval(row['CommonActs'])

    # Create CITES relationship with properties
    cites_relation = Relationship(case_node, "CITES", precedent_node,
                                  CommonIPC=common_ipc_list,
                                  CommonCRPC=common_crpc_list,
                                  CommonCPC=common_cpc_list,
                                  CommonActs=common_acts_list)

    # Merge the relationship in the Neo4j database
    graph.merge(cites_relation, "Case", "CaseID")

  except KeyError:
    print("Keyerror")
    count += 1

# Print a success message
print("Graph created successfully!")
print(count)
